In [2]:
# Train multiple images per person
# Find and recognize faces in an image using a SVC with scikit-learn
# acquired from: https://github.com/ageitgey/face_recognition/blob/master/examples/face_recognition_svm.py
# modified to fit personal use

# set arguments
args = {}
args["dataset"] = "dataset"
args["detection_method"] = "hog" # choose between 'hog', 'cnn'
args["encodings"] = "encodings.pickle"
args["save_location"] = args["encodings"]
print("[INFO] Imports & Arguments set!")

"""
Structure:
        <train_dir>/
            <person_1>/
                <person_1_face-1>.jpg
                <person_1_face-2>.jpg
                .
                .
                <person_1_face-n>.jpg
           <person_2>/
                <person_2_face-1>.jpg
                <person_2_face-2>.jpg
                .
                .
                <person_2_face-n>.jpg
            .
            .
            <person_n>/
                <person_n_face-1>.jpg
                <person_n_face-2>.jpg
                .
                .
                <person_n_face-n>.jpg
"""

import pickle
import face_recognition
from sklearn import svm
import os

# Training the SVC classifier

# The training data would be all the face encodings from all the known images and the labels are their names
knownEncodings = []
knownNames = []

# Training directory
train_dir = os.listdir('./dataset/train/')

# Loop through each person in the training directory
for (i, person) in enumerate(train_dir):
    print("[INFO] processing person [{}/{}]".format(i + 1, len(train_dir)), person)
    pix = os.listdir("./dataset/train/" + person)
    # Loop through each training image for the current person
    for person_img in pix:
        print("[INFO] training using ", person_img)
        # Get the face encodings for the face in each image file
        face = face_recognition.load_image_file("./dataset/train/" + person + "/" + person_img)
        face_bounding_boxes = face_recognition.face_locations(face)

        #If training image contains exactly one face
        if len(face_bounding_boxes) == 1:
            face_enc = face_recognition.face_encodings(face)[0]
            # Add face encoding for current image with corresponding label (name) to the training data
            knownEncodings.append(face_enc)
            knownNames.append(person)
        else:
            print(person + "/" + person_img + " was skipped and can't be used for training because detected {} faces. ".format(len(face_bounding_boxes)))

# # dump the facial encodings + names to disk
# print("[INFO] serializing encodings...")
# data = {"encodings": knownEncodings, "names": knownNames}
# f = open(args["encodings"], "wb")
# f.write(pickle.dumps(data))
# f.close()
# print("[INFO] Serialization Complete!")

# Create and train the SVC classifier
model = svm.SVC(gamma='scale')
model.fit(knownEncodings,knownNames)
f = open(args["save_location"], "wb")
f.write(pickle.dumps(model))
f.close()
print("Model successfully dumped to " + args["save_location"])

[INFO] Imports & Arguments set!
[INFO] processing person [1/30] Roh_Moo-hyun
[INFO] training using  Roh_Moo-hyun_0004.jpg
Roh_Moo-hyun/Roh_Moo-hyun_0004.jpg was skipped and can't be used for training because detected 2 faces. 
[INFO] training using  Roh_Moo-hyun_0008.jpg
[INFO] training using  Roh_Moo-hyun_0010.jpg
[INFO] training using  Roh_Moo-hyun_0009.jpg
[INFO] training using  Roh_Moo-hyun_0003.jpg
[INFO] training using  Roh_Moo-hyun_0005.jpg
[INFO] training using  Roh_Moo-hyun_0007.jpg
[INFO] training using  Roh_Moo-hyun_0002.jpg
Roh_Moo-hyun/Roh_Moo-hyun_0002.jpg was skipped and can't be used for training because detected 3 faces. 
[INFO] training using  Roh_Moo-hyun_0006.jpg
[INFO] training using  Roh_Moo-hyun_0001.jpg
[INFO] processing person [2/30] Gloria_Macapagal_Arroyo
[INFO] training using  Gloria_Macapagal_Arroyo_0007.jpg
[INFO] training using  Gloria_Macapagal_Arroyo_0009.jpg
[INFO] training using  Gloria_Macapagal_Arroyo_0006.jpg
[INFO] training using  Gloria_Macapagal

In [6]:
# import required packages to run
from imutils import paths # to get paths
import cv2 # for face detection
import os # for file system access
import pickle # for storing embeddings
import face_recognition # for embeddings
from matplotlib import pyplot as plt

datasetPathAbs = os.path.abspath(args["dataset"])

if not os.path.exists(args["dataset"]):
    print("[WARN] ", datasetPathAbs, "folder does not exist!")
    print("Create a folder there, and put subfolders named as the person's name (e.g. 'Long_Shun') and put images inside!")
    args["start_check"] = "false"
elif not os.access('.', os.W_OK):
    print("[ERROR] Insufficient permission to write to current directory. You might want to check write permissions of the folder.")
    args["start_check"] = "false"
else:
    print("[INFO] Imports & Path set successfully!")
    args["start_check"] = "true"

[INFO] Imports & Path set successfully!


## Step 1.2: Detect & encode faces

Detect face, encode them into the system, and then store the encodings inside the .pickle file

In [7]:
# set arguments
args = {}
args["dataset"] = "dataset\dataset-full-res"
args["detection_method"] = "hog" # choose between 'hog', 'cnn'
args["encodings"] = "encodings.pickle"
args["start_check"] = "false"

print("[NOTE] Arguments set successfully")

# get input image paths
print("[NOTE] Quantifying Faces")
imagePaths = list(paths.list_images(args["dataset"]))
if not imagePaths:
    print("[ERROR] No image found in '", datasetPathAbs, "' did you forgot to put the images in the dataset folder?")
else:
    # initialize the list of known encodings and known names
    knownEncodings = []
    knownNames = []

    # FOR DEBUGGING
    #print(imagePaths)

    # loop over the image paths
    for (i, imagePath) in enumerate(imagePaths):
        # extract the person name from the image path
        print("[INFO] processing image [{}/{}]".format(i + 1, len(imagePaths)), imagePath)
        name = imagePath.split(os.path.sep)[-2]

        # load the input image and convert it from RGB (OpenCV ordering)
        # to dlib ordering (RGB)
        image = cv2.imread(imagePath)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # detect the (x, y)-coordinates of the bounding boxes
        # corresponding to each face in the input image
        boxes = face_recognition.face_locations(rgb, model=args["detection_method"])
        print(len(boxes))
        if len(boxes) == 0:
            print("[WARN] Can't find faces! ", imagePath)
            print("This image will be excluded from encoding, replace/delete the image to remove this warning.")
        elif len(boxes) > 1:
            print("[WARN] Multiple faces found! ", imagePath)
            print("This image will be excluded from encoding, replace/delete the image to remove this warning")
        else:
            # compute the facial embedding for the face
            encodings = face_recognition.face_encodings(rgb, boxes)

            # loop over the encodings
            for encoding in encodings:
                # add each encoding + name to our set of known names and
                # encodings
                knownEncodings.append(encoding)
                knownNames.append(name)
                
        for (top, right, bottom, left) in boxes:
            # draw the predicted face name on the image
            cv2.rectangle(rgb, (left, top), (right, bottom), (255, 0, 0), 1)
        
        # error checks
        plt.imshow(rgb)
        plt.title(imagePath)
        plt.show()

    # dump the facial encodings + names to disk
    print("[INFO] serializing encodings...")
    data = {"encodings": knownEncodings, "names": knownNames}
    f = open(args["encodings"], "wb")
    f.write(pickle.dumps(data))
    f.close()
    print("[INFO] Serialization Complete!")
    
    # remove duplicates from list
    dedupedNames = []
    [dedupedNames.append(x) for x in knownNames if x not in dedupedNames]
    
    print("Serialized people: ", len(dedupedNames))
    print("Serialized names: ", dedupedNames)
    print("You can now open 2_recognize_faces_in_image Jupyter Notebook to detect the faces of the above person!")

[NOTE] Arguments set successfully
[NOTE] Quantifying Faces
[ERROR] No image found in ' /home/naina/Downloads/face-recognition-low-res-main/knn-version/dataset ' did you forgot to put the images in the dataset folder?


In [8]:
# from fdet import MTCNN

# detector = MTCNN()

# # get input image paths
# print("[NOTE] Quantifying Faces")
# imagePaths = list(paths.list_images(args["dataset"]))

# # initialize the list of known encodings and known names
# knownEncodings = []
# knownNames = []

# # FOR DEBUGGING
# #print(imagePaths)

# # loop over the image paths
# for (i, imagePath) in enumerate(imagePaths):
#     # extract the person name from the image path
#     print("[INFO] processing image {}/{} in {}".format(i + 1, len(imagePaths), imagePath))
#     name = imagePath.split(os.path.sep)[-2]

#     # load the input image and convert it from RGB (OpenCV ordering)
#     # to dlib ordering (RGB)
#     image = cv2.imread(imagePath)
#     rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#     # detect the (x, y)-coordinates of the bounding boxes
#     # corresponding to each face in the input image
#     boxes = detector.detect(rgb)
#     # print(boxes)
#     # compute the facial embedding for the face
#     # print("[NOTE]", len(boxes), " faces found in ", imagePath)
#     for box in boxes:
#         boxFormatted = [tuple(box.get('box'))]
#         encodings = face_recognition.face_encodings(rgb,boxFormatted)
#     # loop over the encodings
#     for encoding in encodings:
#         # add each encoding + name to our set of known names and
#         # encodings
#         knownEncodings.append(encoding)
#         knownNames.append(name)

# # dump the facial encodings + names to disk
# print("[INFO] serializing encodings...")
# data = {"encodings": knownEncodings, "names": knownNames}
# f = open(args["encodings"], "wb")
# f.write(pickle.dumps(data))
# f.close()
# print("[INFO] Complete!")

## Done?

Open [Step 2: Recognize faces in image](2_recognize_faces_in_image.ipynb)